In [1]:
#KGAT_746a8c48819a19cbaf8ca0048244831b

In [2]:
!pip install opendatasets
!pip install pandas

Библиотеки

In [17]:
import opendatasets as od
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.utils.validation import check_array

Скачиваем датасет

Для задачи классификации был взят датасет с данными сайта MyAnimeList. В нём содержится информация об аниме-тайтлах: название, жанры, студии, тип (TV/OVA/фильм и т.д.), сезон выхода, демографический таргет, количество эпизодов, число участников в списках (Members) и другие признаки.
Практическая задача: предсказать, будет ли аниме успешным с точки зрения зрителей. Данная задача применима с практической точки зрения для многих компаний, решающих, в какой проект вложить деньги для последующей прибыли от популярности тайтла.

In [4]:
od.download("https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: angrytea
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/syahrulapriansyah2/myanimelist-2025


100%|██████████| 9.57M/9.57M [00:00<00:00, 835MB/s]

Чтение датасета и просмотр столбцов  
Удаляем строки, не имеющие в Members значения  
Задаем условие успешности: тайтл успешный, если у него 75% и больше зрителей: 1 - аниме успешное (собрало много просмотров) 0 - не успешное  
Для признаков возьмем Source, Type, Released_Season, Episodes, Genres, Studios, Demographic  
Пропуски по эпизодам заменяем медианой, по категориальным признакам на Unknown, жанрам и студиям поставим пустые строки  
Создаем список жанров  
Ищем самые популярные жанры и делаем для них колонки  
Список студий делаем  
Ищем популярные студии, не учитывая 'add some'  
Задаем колонки  
Собираем матрицу X из фич и назначаем целевую переменную успешности y  
Делим данные на train и test  
Числовые и бинарные признаки будем масштабировать, а для категориальных используем one-hot encoding

In [5]:
pd.set_option("display.max_columns", None)
df = pd.read_csv("myanimelist-2025/mal_anime.csv")
df.info()

df["Members"] = pd.to_numeric(df["Members"], errors="coerce")
df = df.dropna(subset=["Members"])

th = df["Members"].quantile(0.75)
df["hit"] = (df["Members"] >= th).astype(int)

df["Episodes"] = pd.to_numeric(df["Episodes"], errors="coerce")
m_episodes = df["Episodes"].median()
df["Episodes"] = df["Episodes"].fillna(m_episodes)

for col in ["Source", "Type", "Released_Season", "Demographic"]:
    df[col] = df[col].fillna("Unknown")

for col in ["Genres", "Studios"]: df[col] = df[col].fillna("")

df["Genres_l"] = df["Genres"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])
genre_count = Counter()

for g in df["Genres_l"]:
    genre_count.update(g)

top_g = [i for i, _ in genre_count.most_common(10)]

for g in top_g:
    col_name = f"genre_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))

g_cols = [f"genre_{g}" for g in top_g]
df[g_cols].head()

df["Studios_l"] = df["Studios"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])
studio_count = Counter()
for s in df["Studios_l"]:
  for st in s:
    clean_s = st.strip()
    if "add some" in clean_s.lower(): continue
    studio_count[clean_s] += 1

top_s = [i for i, _ in studio_count.most_common(10)]

for s in top_s:
    col_name = f"studio_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))

studio_cols = [f"studio_{s}" for s in top_s]

num = ["Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature = num + categ

X = df[feature]
y = df["hit"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

pred = ColumnTransformer(transformers=[
    ("num", StandardScaler(),num),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ)
])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19931 entries, 0 to 19930
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   myanimelist_id   19931 non-null  int64  
 1   title            19931 non-null  object 
 2   description      19875 non-null  object 
 3   image            19544 non-null  object 
 4   Type             19534 non-null  object 
 5   Episodes         19549 non-null  object 
 6   Status           19549 non-null  object 
 7   Premiered        6343 non-null   object 
 8   Released_Season  6182 non-null   object 
 9   Released_Year    6182 non-null   float64
 10  Source           19166 non-null  object 
 11  Genres           18270 non-null  object 
 12  Themes           11217 non-null  object 
 13  Studios          19549 non-null  object 
 14  Producers        19549 non-null  object 
 15  Demographic      6522 non-null   object 
 16  Duration         19549 non-null  object 
 17  Rating      

Строим бейзлайн  


Затем обучается LogisticRegression со стандартными настройками.  
После обучения на X_train, y_train считаю предсказания на тесте и оцениваю качество по метрикам Accuracy, F1-score (для класса hit), ROC-AUC, а также вывожу подробный classification_report по каждому классу

Получаем Accuracy ≈ 0.77, что чуть выше, чем доля класса 0 в выборке (около 0.75). Это значит, что модель использует информацию из признаков и работает немного лучше, чем тривиальный классификатор, всегда предсказывающий «не hit».
По целевому классу hit метрики заметно хуже: F1 ≈ 0.26 при precision ≈ 0.63 и recall ≈ 0.17. То есть, если модель всё-таки предсказывает hit, она в большинстве случаев права (достаточно высокая точность), но реальных успешных тайтлов она находит очень мало - всего около 17 % (очень низкая полнота). Это свидетельствует о сильном перекосе в сторону класса 0 и консервативном принятии решения о том, что тайтл является hit  
Значение ROC-AUC ≈ 0.71 показывает, что по непривязанным к порогу вероятностям модель различает классы значительно лучше случайного угадывания (0.5), но при стандартном пороге 0.5 баланс между precision и recall для класса hit остаётся неудовлетворительным

In [6]:
log_base = Pipeline(steps=[
    ("pred", pred),
    ("classifier", LogisticRegression(max_iter=1000, n_jobs=-1, random_state=42))
])

log_base.fit(X_train, y_train)

y_pred_log = log_base.predict(X_test)
y_proba_log = log_base.predict_proba(X_test)[:, 1]

print("бейзлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("F1-score:", f1_score(y_test, y_pred_log))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_log))
print()
print(classification_report(y_test, y_pred_log))

бейзлайн
Accuracy: 0.7668049792531121
F1-score: 0.26246719160104987
ROC-AUC: 0.7057600492838443

              precision    recall  f1-score   support

           0       0.78      0.97      0.86       903
           1       0.63      0.17      0.26       302

    accuracy                           0.77      1205
   macro avg       0.70      0.57      0.56      1205
weighted avg       0.74      0.77      0.71      1205



1 гипотеза - подбираем гиперпараметры с помощью GridSearchCV  
В сетке перебираю коэффициент регуляризации C и параметр class_weigh, фиксируя L2-регуляризацию. В качестве целевой метрики использую f1 по классу hit, кросс-валидация - 3-кратная. После подбора параметров обучаю лучшую модель на train, считаю предсказания на тесте

GridSearchCV выбирает конфигурацию с L2-регуляризацией, C = 0.1 и class_weight='balanced'  
Accuracy падает с 0.77 до 0.67, зато F1 по классу hit заметно растёт с 0.26 до 0.47, а recall для hit увеличивается примерно с 0.17 до 0.59. Это означает, что модель стала гораздо лучше меньше пропускать хиты, пусть и ценой большего числа ошибок по классу 0 и общего снижения Accuracy. ROC-AUC остаётся примерно на том же уровне 0.70, то есть общая способность разграничивать классы по вероятностям не ухудшилась, меняется именно баланс ошибок между классами

In [7]:
param_grid_log1 = {
    "classifier__penalty": ["l2"],
    "classifier__C": [0.01, 0.1, 1.0, 3.0, 10.0],
    "classifier__class_weight": [None, "balanced"],
}

log_clf1 = Pipeline(steps=[
    ("pred", pred),
    ("classifier", LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        random_state=42,
        solver="lbfgs"
    ))
])

grid_log1 = GridSearchCV(
    estimator=log_clf1,
    param_grid=param_grid_log1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_log1.fit(X_train, y_train)

print("Лучшие параметры:", grid_log1.best_params_)
print("Лучший F1 на CV:", grid_log1.best_score_)

best_log1 = grid_log1.best_estimator_
y_pred_log1 = best_log1.predict(X_test)
y_proba_log1 = best_log1.predict_proba(X_test)[:, 1]

print("\nгипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_log1))
print("F1-score:", f1_score(y_test, y_pred_log1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_log1))
print(classification_report(y_test, y_pred_log1))


Лучшие параметры: {'classifier__C': 0.1, 'classifier__class_weight': 'balanced', 'classifier__penalty': 'l2'}
Лучший F1 на CV: 0.48206640511691146

гипотеза 1
Accuracy: 0.6688796680497925
F1-score: 0.4701195219123506
ROC-AUC: 0.7041099205738047
              precision    recall  f1-score   support

           0       0.83      0.70      0.76       903
           1       0.39      0.59      0.47       302

    accuracy                           0.67      1205
   macro avg       0.61      0.64      0.61      1205
weighted avg       0.72      0.67      0.69      1205



2 гипотеза  
На основе log_Episodes, бинарных жанров и студий заново формирую матрицу признаков и разбиваю данные на train/test. Строю новый пайплайн с тем же препроцессингом (масштабирование чисел, one-hot для категорий) и логистической регрессией. С помощью GridSearchCV подбираю C и class_weight по метрике F1 для класса hit, обучаю лучшую модель и оцениваю качество на тесте (Accuracy, F1, ROC-AUC), чтобы понять, даёт ли логарифмирование эпизодов прирост по сравнению с предыдущей гипотезой

GridSearch снова выбирает те же оптимальные гиперпараметры, что и в гипотезе 1  
Качество на тесте остаётся на очень близком уровне: F1 по классу hit ≈ 0.46 против 0.47 в предыдущей гипотезе, recall для hit немного падает (0.57 вместо 0.59), Accuracy и ROC-AUC почти не меняются. Это показывает, что логарифмирование количества эпизодов в текущей постановке не даёт заметного дополнительного выигрыша по сравнению с уже настроенной логистической регрессией с балансировкой классов

In [8]:
df2 = df.copy()
df2["log_Episodes"] = np.log1p(df2["Episodes"])

num2 = ["log_Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature2 = num2 + categ

X2 = df2[feature2]
y2 = df2["hit"]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,test_size=0.2,random_state=42, stratify=y2)

pred2 = ColumnTransformer(transformers=[
    ("num2", StandardScaler(), num2),
    ("categ", OneHotEncoder(handle_unknown="ignore"), categ),
])

log_clf2 = Pipeline(steps=[
    ("pred", pred2),
    ("classifier", LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        random_state=42
    ))
])

grid_log2 = GridSearchCV(
    estimator=log_clf2,
    param_grid=param_grid_log1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_log2.fit(X2_train,y2_train)

print("Лучшие параметры:", grid_log2.best_params_)
print("Лучший F1 на CV:", grid_log2.best_score_)

best_log2 = grid_log2.best_estimator_
y_pred_log2 = best_log2.predict(X2_test)
y_proba_log2 = best_log2.predict_proba(X2_test)[:, 1]

print("\nЛогистическая регрессия + class_weight/С + log(Episodes)")
print("Accuracy:", accuracy_score(y2_test, y_pred_log2))
print("F1-score:", f1_score(y2_test, y_pred_log2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_log2))
print(classification_report(y2_test, y_pred_log2))

Лучшие параметры: {'classifier__C': 0.1, 'classifier__class_weight': 'balanced', 'classifier__penalty': 'l2'}
Лучший F1 на CV: 0.47723671193058953

Логистическая регрессия + class_weight/С + log(Episodes)
Accuracy: 0.6655601659751037
F1-score: 0.46194926568758343
ROC-AUC: 0.7013267034828716
              precision    recall  f1-score   support

           0       0.83      0.70      0.76       903
           1       0.39      0.57      0.46       302

    accuracy                           0.67      1205
   macro avg       0.61      0.63      0.61      1205
weighted avg       0.72      0.67      0.68      1205



Третья гипотеза  
Пересчитываю самые частые жанры и студии, создаю бинарные признаки для top-20, формирую новый набор числовых признаков и категориальных. Далее заново делю данные на train/test, настраиваю ColumnTransformer (масштабирование числовых признаков и one-hot-кодирование категориальных) и обучаю логистическую регрессию

GridSearch снова выбирает конфигурацию с C = 0.1 и class_weight='balanced'. На тесте F1 по классу hit остаётся на уровне ≈0.47, как и в первой гипотезе, но общая Accuracy немного растёт (с ~0.67 до ~0.68), при этом recall по hit держится около 0.57. ROC-AUC почти не меняется. Это означает, что добавление признаков для более широкого набора жанров и студий немного улучшает общую точность, но в целом не даёт серьёзного прироста качества по сравнению с уже настроенной моделью

In [9]:
top_s1 = [i for i, _ in studio_count.most_common(20)]
for s in top_s1:
    col_name = f"studio_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))

studio_cols1 = [f"studio_{s}" for s in top_s1]

top_g1 = [i for i, _ in genre_count.most_common(20)]

for g in top_g1:
    col_name = f"genre_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))

g_cols1 = [f"genre_{g}" for g in top_g1]

num = ["Episodes"] + g_cols1 + studio_cols1
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature = num + categ

X = df[feature]
y = df["hit"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

pred = ColumnTransformer(transformers=[("num", StandardScaler(), num), ("categ", OneHotEncoder(handle_unknown="ignore"), categ)])

model_n = Pipeline(steps=[
    ("pred", pred),
    ("classifier", LogisticRegression(
        max_iter=1000,
        n_jobs=-1,
        random_state=42
    ))
])

grid_search = GridSearchCV(
    estimator=model_n,
    param_grid=param_grid_log1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
print("Лучший F1 на кросс-валидации:", grid_search.best_score_)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc = roc_auc_score(y_test, y_proba)

print("top-20 жанров/студий")
print("Accuracy:", acc)
print("F1-score:", f1)
print("ROC-AUC:", roc)
print(classification_report(y_test, y_pred))

Лучшие параметры: {'classifier__C': 0.1, 'classifier__class_weight': 'balanced', 'classifier__penalty': 'l2'}
Лучший F1 на кросс-валидации: 0.4765046512668795
top-20 жанров/студий
Accuracy: 0.6771784232365146
F1-score: 0.46785225718194257
ROC-AUC: 0.702586301731535
              precision    recall  f1-score   support

           0       0.83      0.71      0.77       903
           1       0.40      0.57      0.47       302

    accuracy                           0.68      1205
   macro avg       0.61      0.64      0.62      1205
weighted avg       0.72      0.68      0.69      1205



Реализую собственную модель  
Модель поддерживает L2-регуляризацию и учёт дисбаланса классов через class_weight="balanced"  
В fit привожу данные к плотному float-массиву, кодирую таргет в 0/1,инициализирую веса, оптимизирую логистическую функцию потерь с L2-штрафом с помощью полного градиентного спуска и sample_weight, после обучения подбираю оптимальный порог вероятности по F1 на обучающей выборке и сохраняю его в self.threshold_  
В predict_proba возвращаю вероятности двух классов,
а в predict перевожу их в метки классов, используя подобранный порог и обратно маппя 0/1 в исходные значения классов

In [15]:
class MyLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(
        self,
        lr=0.1,
        n_iter=1000,
        penalty="l2",        # 'none' или 'l2'
        C=1.0,
        class_weight=None,   # None или 'balanced'
        fit_intercept=True,
        random_state=None
    ):
        self.lr = lr
        self.n_iter = n_iter
        self.penalty = penalty
        self.C = C
        self.class_weight = class_weight
        self.fit_intercept = fit_intercept
        self.random_state = random_state
        self.threshold_ = 0.5

    def _sigmoid(self, z):
        z = np.clip(z, -30, 30)
        return 1.0 / (1.0 + np.exp(-z))

    def fit(self, X, y):
        X = check_array(X, accept_sparse=False, dtype=float)
        y = np.asarray(y)

        self.classes_ = np.unique(y)
        if self.classes_.shape[0] != 2:
            raise ValueError("MyLogisticRegression поддерживает только бинарную классификацию")

        y_bin = (y == self.classes_[1]).astype(float)

        n_samples, n_features = X.shape
        rng = np.random.RandomState(self.random_state)

        self.w_ = rng.normal(scale=0.01, size=n_features)
        self.b_ = 0.0

        if self.class_weight is None:
            sample_weight = np.ones(n_samples)
        elif self.class_weight == "balanced":
            n_pos = y_bin.sum()
            n_neg = n_samples - n_pos
            w0 = n_samples / (2.0 * n_neg)
            w1 = n_samples / (2.0 * n_pos)
            sample_weight = np.where(y_bin == 1.0, w1, w0)
        else:
            raise ValueError("class_weight должен быть None или 'balanced'")

        if self.penalty == "l2":
            lam = 1.0 / self.C
        elif self.penalty in ["none", None]:
            lam = 0.0
        else:
            raise ValueError("поддерживаются только penalty='l2' или 'none'")

        for _ in range(self.n_iter):
            z = X @ self.w_ + self.b_
            p = self._sigmoid(z)
            error = (p - y_bin) * sample_weight
            grad_w = X.T @ error / n_samples + lam * self.w_
            grad_b = error.mean()
            self.w_ -= self.lr * grad_w
            self.b_ -= self.lr * grad_b

        z_train = X @ self.w_ + self.b_
        p_train = self._sigmoid(z_train)

        best_thr = 0.5
        best_f1 = -1.0
        for thr in np.linspace(0.1, 0.9, 17):
            y_pred_thr = (p_train >= thr).astype(int)
            f1 = f1_score(y_bin, y_pred_thr)
            if f1 > best_f1:
                best_f1 = f1
                best_thr = thr

        self.threshold_ = best_thr
        return self

    def predict_proba(self, X):
        X = check_array(X, accept_sparse=False, dtype=float)
        z = X @ self.w_ + self.b_
        p1 = self._sigmoid(z)
        p0 = 1.0 - p1
        return np.vstack([p0, p1]).T

    def predict(self, X):
        proba = self.predict_proba(X)
        y_bin_pred = (proba[:, 1] >= self.threshold_).astype(int)
        return self.classes_[y_bin_pred]

Бейзлайн собственной реализации  
Сначала раскладываю строковые поля Genres и Studios в списки, считаю частоты и выбираю top-10 самых популярных жанров и студий. Для них создаю бинарные признаки вида genre_X и studio_Y, которые показывают наличие соответствующего тега у тайтла. Далее формирую набор числовых признаков (Episodes + бинарные жанры и студии) и категориальных (Source, Type, Released_Season, Demographic), собираю матрицу признаков X и целевую переменную hit, и делю данные на обучающую и тестовую выборки со стратификацией по классу. Через ColumnTransformer масштабирую числовые признаки и one-hot кодирую категориальные, после чего в пайплайне обучаю свою MyLogisticRegression с L2-регуляризацией

У sklearn-модели выше accuracy (≈0.77 против ≈0.65) и ROC-AUC (≈0.71 против ≈0.66), то есть она в среднем лучше ранжирует объекты по вероятности класса hit и реже ошибается, если смотреть на долю верных ответов  
Однако моя реализация даёт гораздо более высокий F1 по классу hit (≈0.46 против ≈0.26) за счёт другого баланса precision/recall  
sklearn-модель сильно осторожничает и почти всегда предсказывает класс 0 (recall по hit ≈0.17)  
Моя модель, благодаря подбору порога по F1, чаще предсказывает класс 1 (recall по hit ≈0.60), жертвуя точностью (precision падает с ≈0.63 до ≈0.37)  
В результате кастомная модель хуже по общей точности и ROC-AUC, но лучше решает задачу “находить как можно больше успешных тайтлов” и даёт более сбалансированные метрики для редкого класса hit.

In [18]:
df["Genres_l"] = df["Genres"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])
genre_count = Counter()

for g in df["Genres_l"]:
    genre_count.update(g)

top_g = [i for i, _ in genre_count.most_common(10)]

for g in top_g:
    col_name = f"genre_{g}"
    df[col_name] = df["Genres_l"].apply(lambda lst: int(g in lst))

g_cols = [f"genre_{g}" for g in top_g]
df[g_cols].head()

df["Studios_l"] = df["Studios"].apply(lambda x: [i.strip() for i in str(x).split(",") if i.strip() != ""])
studio_count = Counter()
for s in df["Studios_l"]:
  for st in s:
    clean_s = st.strip()
    if "add some" in clean_s.lower(): continue
    studio_count[clean_s] += 1

top_s = [i for i, _ in studio_count.most_common(10)]

for s in top_s:
    col_name = f"studio_{s}"
    df[col_name] = df["Studios_l"].apply(lambda lst: int(s in lst))

studio_cols = [f"studio_{s}" for s in top_s]

num = ["Episodes"] + g_cols + studio_cols
categ = ["Source", "Type", "Released_Season", "Demographic"]
feature = num + categ

X = df[feature]
y = df["hit"]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

pred = ColumnTransformer(transformers=[("num", StandardScaler(),num),("categ", OneHotEncoder(handle_unknown="ignore"), categ)])

log_base = Pipeline(steps=[
    ("pred", pred),
    ("classifier", MyLogisticRegression(
        lr=0.1,
        n_iter=1000,
        penalty="l2",
        C=1.0,
        class_weight=None,
        random_state=42
    ))
])

log_base.fit(X_train, y_train)

y_pred_log = log_base.predict(X_test)
y_proba_log = log_base.predict_proba(X_test)[:, 1]

print("бейзлайн")
print("Accuracy:", accuracy_score(y_test, y_pred_log))
print("F1-score:", f1_score(y_test, y_pred_log))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_log))
print(classification_report(y_test, y_pred_log))

бейзлайн
Accuracy: 0.6481327800829876
F1-score: 0.45918367346938777
ROC-AUC: 0.6638394461434659
              precision    recall  f1-score   support

           0       0.83      0.67      0.74       903
           1       0.37      0.60      0.46       302

    accuracy                           0.65      1205
   macro avg       0.60      0.63      0.60      1205
weighted avg       0.72      0.65      0.67      1205



1 гипотеза

лучший набор оказался: C = 10, penalty = 'l2', class_weight = None. То есть модели выгоднее слабее регуляризоваться (большой C) и не вводить явный баланс классов - дисбаланс по hit мы уже частично компенсируем обучением и подбором порога  
По сравнению с собственным бейзлайном гипотеза 1 даёт небольшое, но стабильное улучшение всех метрик: accuracy выросла до ≈ 0.652, F1 до ≈ 0.466, ROC-AUC до ≈ 0.682  
То есть модель стала чуть лучше разделять классы по вероятностям и чуть лучше находить хиты: recall по классу 1 вырос до ≈ 0.61, precision до ≈ 0.38, F1 стало выше  
Если сравнивать с логистической регрессией из sklearn со 2-й гипотезы, то качество очень близко: чуть больший F1 на классе hit (≈ 0.466 против ≈ 0.462), но проигрывает по Accuracy и ROC-AUC (≈ 0.652 / 0.682 против ≈ 0.666 / 0.701 у sklearn).

In [19]:
my_log_clf1 = Pipeline(steps=[
    ("pred", pred),
    ("classifier", MyLogisticRegression(
        lr=0.1,
        n_iter=1000,
        penalty="l2",
        C=1.0,
        class_weight=None,
        random_state=42
    ))
])

param_grid_my1 = {
    "classifier__penalty": ["l2"],
    "classifier__C": [0.01, 0.1, 1.0, 3.0, 10.0],
    "classifier__class_weight": [None, "balanced"],
}

grid_my1 = GridSearchCV(
    estimator=my_log_clf1,
    param_grid=param_grid_my1,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my1.fit(X_train, y_train)

print("Лучшие параметры:", grid_my1.best_params_)
print("Лучший F1 на CV:", grid_my1.best_score_)

best_my_log1 = grid_my1.best_estimator_

y_pred_my1 = best_my_log1.predict(X_test)
y_proba_my1 = best_my_log1.predict_proba(X_test)[:, 1]

print("\nгипотеза 1")
print("Accuracy:", accuracy_score(y_test, y_pred_my1))
print("F1-score:", f1_score(y_test, y_pred_my1))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my1))
print(classification_report(y_test, y_pred_my1))


Лучшие параметры: {'classifier__C': 10.0, 'classifier__class_weight': None, 'classifier__penalty': 'l2'}
Лучший F1 на CV: 0.47077361322449973

гипотеза 1
Accuracy: 0.6522821576763486
F1-score: 0.4662420382165605
ROC-AUC: 0.6819761941431431
              precision    recall  f1-score   support

           0       0.84      0.67      0.74       903
           1       0.38      0.61      0.47       302

    accuracy                           0.65      1205
   macro avg       0.61      0.64      0.60      1205
weighted avg       0.72      0.65      0.67      1205



2 гипотеза

GridSearch снова выбирает C = 10 и без явного взвешивания классов (class_weight=None), то есть модели выгодно слабое L2-наказание и обучение на исходном дисбалансе  
По сравнению с бейзлайном метрики немного улучшаются: Accuracy растёт до ≈ 0.654, F1 до ≈ 0.47, ROC-AUC до ≈ 0.678. Модель по-прежнему даёт recall по классу hit ≈ 0.61 при precision ≈ 0.38, но F1 становится чуть выше, чем в гипотезе 1  
В сравнении со 2-й гипотезой sklearn собственная реализация снова немного выигрывает в F1 по редкому классу hit, но уступает по общей точности и AUC. То есть кастомная модель остаётся чуть более агрессивной по отношению к классу 1 и ориентирована на максимизацию F1, а не на глобальную точность

In [20]:
my_log_clf2 = Pipeline(steps=[
    ("pred", pred2),
    ("classifier", MyLogisticRegression(
        lr=0.1,
        n_iter=1000,
        penalty="l2",
        C=1.0,
        class_weight=None,
        random_state=42
    ))
])

param_grid_my2 = {
    "classifier__penalty": ["l2"],
    "classifier__C": [0.01, 0.1, 1.0, 3.0, 10.0],
    "classifier__class_weight": [None, "balanced"],
}

grid_my2 = GridSearchCV(
    estimator=my_log_clf2,
    param_grid=param_grid_my2,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my2.fit(X2_train, y2_train)

print("Лучшие параметры:", grid_my2.best_params_)
print("Лучший F1 на CV:", grid_my2.best_score_)

best_my_log2 = grid_my2.best_estimator_

y_pred_my2 = best_my_log2.predict(X2_test)
y_proba_my2 = best_my_log2.predict_proba(X2_test)[:, 1]

print("\nгипотеза 2")
print("Accuracy:", accuracy_score(y2_test, y_pred_my2))
print("F1-score:", f1_score(y2_test, y_pred_my2))
print("ROC-AUC:", roc_auc_score(y2_test, y_proba_my2))
print()
print(classification_report(y2_test, y_pred_my2))


Лучшие параметры: {'classifier__C': 10.0, 'classifier__class_weight': None, 'classifier__penalty': 'l2'}
Лучший F1 на CV: 0.47214686548019885

гипотеза 2
Accuracy: 0.653941908713693
F1-score: 0.4701397712833545
ROC-AUC: 0.6778655401788007

              precision    recall  f1-score   support

           0       0.84      0.67      0.74       903
           1       0.38      0.61      0.47       302

    accuracy                           0.65      1205
   macro avg       0.61      0.64      0.61      1205
weighted avg       0.72      0.65      0.67      1205



3 гипотеза

GridSearch снова выбирает C = 10 и class_weight=None, а качество на тесте практически совпадает с гипотезой 1: Accuracy ≈ 0.65, F1 ≈ 0.47, ROC-AUC ≈ 0.682. Уточнение жанров/студий за счёт расширения до top-20 заметного прироста метрик не даёт - модель по-прежнему остаётся более чувствительной к классу hit (recall ≈ 0.61 при сравнительно низком precision ≈ 0.38), но баланс качество/сложность признаков почти не меняется

In [21]:
my_log_clf3 = Pipeline(steps=[
    ("pred", pred),
    ("classifier", MyLogisticRegression(
        lr=0.1,
        n_iter=1000,
        penalty="l2",
        C=1.0,
        class_weight=None,
        random_state=42
    ))
])

param_grid_my3 = {
    "classifier__penalty": ["l2"],
    "classifier__C": [0.01, 0.1, 1.0, 3.0, 10.0],
    "classifier__class_weight": [None, "balanced"],
}

grid_my3 = GridSearchCV(
    estimator=my_log_clf3,
    param_grid=param_grid_my3,
    scoring="f1",
    cv=3,
    n_jobs=-1
)

grid_my3.fit(X_train, y_train)

print("Лучшие параметры:", grid_my3.best_params_)
print("Лучший F1 на CV:", grid_my3.best_score_)

best_my_log3 = grid_my3.best_estimator_

y_pred_my3 = best_my_log3.predict(X_test)
y_proba_my3 = best_my_log3.predict_proba(X_test)[:, 1]

print("\ntop-20 жанров/студий")
print("Accuracy:", accuracy_score(y_test, y_pred_my3))
print("F1-score:", f1_score(y_test, y_pred_my3))
print("ROC-AUC:", roc_auc_score(y_test, y_proba_my3))
print("\nОтчёт по классам:")
print(classification_report(y_test, y_pred_my3))

Лучшие параметры: {'classifier__C': 10.0, 'classifier__class_weight': None, 'classifier__penalty': 'l2'}
Лучший F1 на CV: 0.47077361322449973

top-20 жанров/студий
Accuracy: 0.6522821576763486
F1-score: 0.4662420382165605
ROC-AUC: 0.6819761941431431

Отчёт по классам:
              precision    recall  f1-score   support

           0       0.84      0.67      0.74       903
           1       0.38      0.61      0.47       302

    accuracy                           0.65      1205
   macro avg       0.61      0.64      0.60      1205
weighted avg       0.72      0.65      0.67      1205



Простой бейзлайн из sklearn даёт высокую accuracy за счёт перекоса в сторону неуспешного класса, но почти не видит хиты. После подбора гиперпараметров и учета дисбаланса классов удалось заметно поднять F1 по классу hit и recall, пожертвовав частью общей точности. Добавление логарифма по числу эпизодов и расширение набора бинарных признаков до top-20 жанров и студий дало лишь небольшие, косметические улучшения, без качественного скачка. Собственная реализация логистической регрессии воспроизводит те же тенденции и по метрикам ведет себя близко к библиотечной версии: где-то чуть выигрывает в F1 по редкому классу, где-то немного уступает по accuracy и ROC-AUC. В целом можно считать, что модель работает адекватно, а ключевую роль снова играет не столько выбор алгоритма, сколько грамотная постановка таргета, обработка дисбаланса и аккуратная работа с признаками